<a href="https://colab.research.google.com/github/thmswhelan/UCDPA_ThomasWhelan/blob/main/PropertyPricesIreland.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/thmswhelan/UCDPA_ThomasWhelan/main/Property_Price_Register_Ireland-28-05-2021.csv"

df = pd.read_csv(url)

In [2]:
df.head()

,SALE_DATE,ADDRESS,POSTAL_CODE,COUNTY,SALE_PRICE,IF_MARKET_PRICE,IF_VAT_EXCLUDED,PROPERTY_DESC,PROPERTY_SIZE_DESC
0,2010-01-01,"5 Braemor Drive, Churchtown, Co.Dublin",NaN,Dublin,343000.0,0,0,Second-Hand Dwelling house /Apartment,NaN
1,2010-01-03,"134 Ashewood Walk, Summerhill Lane, Portlaoise",NaN,Laois,185000.0,0,1,New Dwelling house /Apartment,greater than or equal to 38 sq metres and less...
2,2010-01-04,"1 Meadow Avenue, Dundrum, Dublin 14",NaN,Dublin,438500.0,0,0,Second-Hand Dwelling house /Apartment,NaN
3,2010-01-04,"1 The Haven, Mornington",NaN,Meath,400000.0,0,0,Second-Hand Dwelling house /Apartment,NaN
4,2010-01-04,"11 Melville Heights, Kilkenny",NaN,Kilkenny,160000.0,0,0,Second-Hand Dwelling house /Apartment,NaN


In [3]:
import numpy as np

In [5]:
df.dtypes

SALE_DATE              object
ADDRESS                object
POSTAL_CODE            object
COUNTY                 object
SALE_PRICE            float64
IF_MARKET_PRICE         int64
IF_VAT_EXCLUDED         int64
PROPERTY_DESC          object
PROPERTY_SIZE_DESC     object
dtype: object

In [6]:
df.info

<bound method DataFrame.info of          SALE_DATE                                         ADDRESS  \
0       2010-01-01          5 Braemor Drive, Churchtown, Co.Dublin   
1       2010-01-03  134 Ashewood Walk, Summerhill Lane, Portlaoise   
2       2010-01-04             1 Meadow Avenue, Dundrum, Dublin 14   
3       2010-01-04                         1 The Haven, Mornington   
4       2010-01-04                   11 Melville Heights, Kilkenny   
...            ...                                             ...   
476740  2021-05-28                GRANGE STORES, GRANGE, MULLINGAR   
476741  2021-05-28               MAIN STREET, LOWER BORRIS, BORRIS   
476742  2021-05-28                         MULTYFARNHAM, WESTMEATH   
476743  2021-05-28             NO 2 RIVERLODGE, SHRONE, GLENGARIFF   
476744  2021-05-28                         RABANE, KILL, COOTEHILL   

       POSTAL_CODE     COUNTY  SALE_PRICE  IF_MARKET_PRICE  IF_VAT_EXCLUDED  \
0              NaN     Dublin    343000.0       

In [7]:
df.describe()

,SALE_PRICE,IF_MARKET_PRICE,IF_VAT_EXCLUDED
count,4.767450e+05,476745.000000,476745.000000
mean,2.590403e+05,0.048984,0.159618
std,8.526791e+05,0.215835,0.366252
min,5.030530e+03,0.000000,0.000000
25%,1.150000e+05,0.000000,0.000000
50%,1.950000e+05,0.000000,0.000000
75%,3.000000e+05,0.000000,0.000000
max,1.701428e+08,1.000000,1.000000


In [8]:
df.value_counts()

SALE_DATE   ADDRESS                                                   POSTAL_CODE  COUNTY  SALE_PRICE  IF_MARKET_PRICE  IF_VAT_EXCLUDED  PROPERTY_DESC                  PROPERTY_SIZE_DESC                                               
2013-12-20  2 Phoenix Park Way, Phoenix Park Racecourse, Castleknock  Dublin 15    Dublin  8000.00     0                1                New Dwelling house /Apartment  greater than or equal to 38 sq metres and less than 125 sq metres    4
2013-11-20  Apt. 9 Red Arches Park, The Coast, Baldoyle               Dublin 13    Dublin  127753.30   0                1                New Dwelling house /Apartment  greater than or equal to 38 sq metres and less than 125 sq metres    3
2013-12-20  77 Rosanule, Phoenix Park Racecourse, Castleknock         Dublin 15    Dublin  8000.00     0                1                New Dwelling house /Apartment  greater than or equal to 38 sq metres and less than 125 sq metres    3
            4 Surehaven Road, Phoenix Park Raceco

In [9]:
np.mean(df['SALE_PRICE'])

259040.30628275065

In [11]:
county_prices = df.groupby('COUNTY')

In [13]:
county_prices.mean()

<ipython-input-13-b9d3177beca3>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  county_prices.mean()


,SALE_PRICE,IF_MARKET_PRICE,IF_VAT_EXCLUDED
COUNTY,,,
Carlow,152975.334509,0.074509,0.153831
Cavan,118877.039582,0.019116,0.149810
Clare,159339.057546,0.032578,0.101588
Cork,228345.737261,0.050900,0.156782
Donegal,117704.793719,0.059119,0.102545
Dublin,411673.133249,0.044035,0.176904
Galway,204995.368932,0.041776,0.118122
Kerry,160922.120121,0.058798,0.107083
Kildare,266907.150575,0.069960,0.284011
